In [1]:
import $cp.`/jars/*`

import $cp.$

In [2]:

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.kafka.clients.producer._

import java.util.Properties

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._

val bootstrapServers = "kafka:9092"
val kafkaProps = new Properties()
kafkaProps.put("bootstrap.servers", bootstrapServers)
kafkaProps.put("key.serializer", "org.apache.kafka.common.serialization.StringSerializer")
kafkaProps.put("value.serializer", "org.apache.kafka.common.serialization.StringSerializer")

val kafkaProducer = new KafkaProducer[String, String](kafkaProps)

def sendToKafka(topic: String, values: Seq[String]): Unit = {
 
  values.foreach { value =>
    val record = new ProducerRecord[String, String](topic, value)
    kafkaProducer.send(record)
  }
  kafkaProducer.flush()
}

def readFromKafkaSpark(topic: String): DataFrame = {
  spark.read
  .format("kafka")
  .option("kafka.bootstrap.servers", bootstrapServers)
  .option("subscribe", topic)
  .load()
  .selectExpr("CAST(value AS STRING) as message")
}

val kafkaTopic = "my-topic-scala"


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.kafka.clients.producer._
import java.util.Properties
spark: SparkSession = org.apache.spark.sql.SparkSession@28004f99
import spark.implicits._
bootstrapServers: String = "kafka:9092"
kafkaProps: Properties = {value.serializer=org.apache.kafka.common.serialization.StringSerializer, bootstrap.servers=kafka:9092, key.serializer=org.apache.kafka.common.serialization.StringSerializer}
res2_7: Object = null
res2_8: Object = null
res2_9: Object = null
kafkaProducer: KafkaProducer[String, String] = org.apache.kafka.clients.producer.KafkaProducer@4e5ed906
defined function sendToKafka
defined function readFromKafkaSpark
kafkaTopic: String = "my-topic-scala"

In [3]:
val data = Seq("hi", "there", "from", "scala")

sendToKafka(kafkaTopic, data)

data: Seq[String] = List("hi", "there", "from", "scala")

In [4]:
readFromKafkaSpark(kafkaTopic).show()

SLF4J: Failed to load class "org.slf4j.impl.StaticMDCBinder".
SLF4J: Defaulting to no-operation MDCAdapter implementation.
SLF4J: See http://www.slf4j.org/codes.html#no_static_mdc_binder for further details.


+-------+
|message|
+-------+
|     hi|
|  there|
|   from|
|  scala|
+-------+

